In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pylab as plt

!pip install dmba
from dmba import classificationSummary

In [14]:
ksi = pd.read_csv('KSI.csv')
ksi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16860 entries, 0 to 16859
Data columns (total 57 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   X                16860 non-null  float64
 1   Y                16860 non-null  float64
 2   INDEX_           16860 non-null  int64  
 3   ACCNUM           16860 non-null  int64  
 4   YEAR             16860 non-null  int64  
 5   DATE             16860 non-null  object 
 6   TIME             16860 non-null  int64  
 7   HOUR             16860 non-null  int64  
 8   STREET1          16860 non-null  object 
 9   STREET2          16860 non-null  object 
 10  OFFSET           16860 non-null  object 
 11  ROAD_CLASS       16860 non-null  object 
 12  DISTRICT         16860 non-null  object 
 13  WARDNUM          16860 non-null  object 
 14  DIVISION         16860 non-null  object 
 15  LATITUDE         16860 non-null  float64
 16  LONGITUDE        16860 non-null  float64
 17  LOCCOORD    

In [15]:
# Combine Latitude and Longitude
ksi['NEW_LATITUDE'] = ksi['LATITUDE'].round(2)
ksi['NEW_LONGITUDE'] = ksi['LONGITUDE'].round(2)
ksi['Combine_Lat_Long'] = ksi['NEW_LATITUDE'].astype(str) + ", " +  ksi['NEW_LONGITUDE'].astype(str)

In [16]:
# Calculate Sum ACCNUM grouped by combined latitude and longitude
ksi['Sum_KSI_Incidents'] = ksi['ACCNUM'].groupby(ksi['Combine_Lat_Long']).transform('nunique')

In [17]:
# Extract only the important ksi variables to combine with HDA dataset
ksi_subset = ksi[['Combine_Lat_Long', 'Sum_KSI_Incidents']]
ksi_subset

,Combine_Lat_Long,Sum_KSI_Incidents
0,"43.66, -79.45",25
1,"43.66, -79.45",25
2,"43.8, -79.2",17
3,"43.8, -79.2",17
4,"43.73, -79.26",14
...,...,...
16855,"43.71, -79.24",3
16856,"43.74, -79.23",8
16857,"43.74, -79.23",8
16858,"43.74, -79.23",8


In [18]:
# Remove the duplicate records so that each row is a unique Combine_Lat_Long
ksi_subset = ksi_subset.drop_duplicates()
ksi_subset

,Combine_Lat_Long,Sum_KSI_Incidents
0,"43.66, -79.45",25
2,"43.8, -79.2",17
4,"43.73, -79.26",14
7,"43.65, -79.42",29
9,"43.7, -79.55",15
...,...,...
16111,"43.71, -79.32",1
16148,"43.64, -79.49",1
16257,"43.76, -79.53",1
16426,"43.7, -79.58",2


In [19]:
hda = pd.read_csv('HazardousDrivingAreas.csv')
hda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2963 entries, 0 to 2962
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Geohash         2963 non-null   object 
 1   GeohashBounds   2963 non-null   object 
 2   Latitude_SW     2963 non-null   float64
 3   Longitude_SW    2963 non-null   float64
 4   Latitude_NE     2963 non-null   float64
 5   Longitude_NE    2963 non-null   float64
 6   Location        2963 non-null   object 
 7   Latitude        2963 non-null   float64
 8   Longitude       2963 non-null   float64
 9   City            2963 non-null   object 
 10  County          0 non-null      float64
 11  State           2963 non-null   object 
 12  Country         2963 non-null   object 
 13  ISO_3166_2      2963 non-null   object 
 14  SeverityScore   2963 non-null   float64
 15  IncidentsTotal  2963 non-null   int64  
 16  UpdateDate      2963 non-null   object 
 17  Version         2963 non-null   f

In [20]:
# Combine Latitude and Longitude
hda['New_Latitude'] = hda['Latitude'].round(2)
hda['New_Longitude'] = hda['Longitude'].round(2)
hda['Combine_Lat_Long'] = hda['New_Latitude'].astype(str) + ", " +  hda['New_Longitude'].astype(str)

In [21]:
# Calculate Sum IncidentsTotal and Average SeverityScore grouped by combined latitude and longitude
hda['Sum_HDA_Incidents'] = hda['IncidentsTotal'].groupby(hda['Combine_Lat_Long']).transform('sum')
hda['Avg_Severity'] = hda['SeverityScore'].groupby(hda['Combine_Lat_Long']).transform('mean')

In [22]:
# Extract only the important hda variables to combine with KSI dataset
hda_subset = hda[['Combine_Lat_Long', 'Sum_HDA_Incidents', 'Avg_Severity']]
hda_subset.sort_values(by='Combine_Lat_Long', ascending=False)

,Combine_Lat_Long,Sum_HDA_Incidents,Avg_Severity
2813,"43.85, -79.19",3,0.032
1605,"43.84, -79.25",23,0.028
1059,"43.84, -79.25",23,0.028
773,"43.83, -79.28",22,0.027
1350,"43.83, -79.26",35,0.081
...,...,...,...
2792,"43.6, -79.5",3,0.007
2294,"43.59, -79.54",52,0.052
1789,"43.59, -79.54",52,0.052
1551,"43.59, -79.54",52,0.052


In [23]:
# Remove the duplicate records so that each row is a unique Combine_Lat_Long
hda_subset = hda_subset.drop_duplicates()
hda_subset

,Combine_Lat_Long,Sum_HDA_Incidents,Avg_Severity
0,"43.77, -79.36",5186,0.023846
1,"43.71, -79.54",5566,0.019563
2,"43.74, -79.43",2307,0.018333
3,"43.74, -79.42",3213,0.019312
4,"43.72, -79.49",2940,0.023400
...,...,...,...
2897,"43.81, -79.31",3,0.008000
2898,"43.8, -79.22",3,0.004000
2929,"43.71, -79.32",3,0.094000
2937,"43.79, -79.39",3,0.008000


In [24]:
# Merge KSI and HDA Datasets
merge_df = pd.merge(ksi_subset, hda_subset, how='left', on='Combine_Lat_Long')

In [25]:
merge_df['Avg_Severity'] = np.where(merge_df['Avg_Severity'].isnull(), 0, merge_df['Avg_Severity'])
merge_df['Sum_HDA_Incidents'] = np.where(merge_df['Sum_HDA_Incidents'].isnull(), 0, merge_df['Sum_HDA_Incidents'])

In [26]:
merge_df.Sum_HDA_Incidents = merge_df.Sum_HDA_Incidents.astype('int')

In [28]:
merge_df.sort_values(by='Sum_HDA_Incidents', ascending=False)

,Combine_Lat_Long,Sum_KSI_Incidents,Sum_HDA_Incidents,Avg_Severity
379,"43.71, -79.54",3,5566,0.019563
46,"43.77, -79.36",4,5186,0.023846
372,"43.74, -79.42",5,3213,0.019312
621,"43.72, -79.49",6,2940,0.023400
252,"43.62, -79.53",12,2375,0.026889
...,...,...,...,...
493,"43.65, -79.58",4,0,0.000000
495,"43.81, -79.34",2,0,0.000000
61,"43.67, -79.54",1,0,0.000000
509,"43.71, -79.31",8,0,0.000000


In [30]:
ksi[['ACCNUM', 'LATITUDE', 'LONGITUDE', 'DISTRICT', 'NEIGHBOURHOOD']].loc[ksi['Combine_Lat_Long'].isin(['43.71, -79.54'])]

,ACCNUM,LATITUDE,LONGITUDE,DISTRICT,NEIGHBOURHOOD
2219,1015222,43.714745,-79.536690,Etobicoke York,Pelmo Park-Humberlea (23)
2220,1015222,43.714745,-79.536690,Etobicoke York,Pelmo Park-Humberlea (23)
2483,995821,43.711245,-79.535190,Etobicoke York,Pelmo Park-Humberlea (23)
2488,995821,43.711245,-79.535190,Etobicoke York,Pelmo Park-Humberlea (23)
2491,995821,43.711245,-79.535190,Etobicoke York,Pelmo Park-Humberlea (23)
2496,995821,43.711245,-79.535190,Etobicoke York,Pelmo Park-Humberlea (23)
14488,8000311536,43.711795,-79.535419,Etobicoke York,Pelmo Park-Humberlea (23)
14489,8000311536,43.711795,-79.535419,Etobicoke York,Pelmo Park-Humberlea (23)
14490,8000311536,43.711795,-79.535419,Etobicoke York,Pelmo Park-Humberlea (23)
14491,8000311536,43.711795,-79.535419,Etobicoke York,Pelmo Park-Humberlea (23)


In [31]:
ksi[['ACCNUM', 'LATITUDE', 'LONGITUDE', 'DISTRICT', 'NEIGHBOURHOOD']].loc[ksi['Combine_Lat_Long'].isin(['43.77, -79.36'])]

,ACCNUM,LATITUDE,LONGITUDE,DISTRICT,NEIGHBOURHOOD
139,919534,43.768745,-79.356890,North York,Henry Farm (53)
140,919534,43.768745,-79.356890,North York,Henry Farm (53)
10109,4002131801,43.769412,-79.364269,North York,Bayview Village (52)
10124,4002131801,43.769412,-79.364269,North York,Bayview Village (52)
10132,4002131801,43.769412,-79.364269,North York,Bayview Village (52)
13608,6002282279,43.767161,-79.363620,North York,Bayview Village (52)
13609,6002282279,43.767161,-79.363620,North York,Bayview Village (52)
14213,7003136062,43.771571,-79.364093,North York,Henry Farm (53)
14214,7003136062,43.771571,-79.364093,North York,Henry Farm (53)
14215,7003136062,43.771571,-79.364093,North York,Henry Farm (53)


In [32]:
ksi[['ACCNUM', 'LATITUDE', 'LONGITUDE', 'DISTRICT', 'NEIGHBOURHOOD']].loc[ksi['Combine_Lat_Long'].isin(['43.74, -79.42'])]

,ACCNUM,LATITUDE,LONGITUDE,DISTRICT,NEIGHBOURHOOD
2189,1013236,43.737545,-79.420590,North York,Bedford Park-Nortown (39)
2190,1013236,43.737545,-79.420590,North York,Bedford Park-Nortown (39)
8643,1295871,43.742343,-79.418292,North York,Bedford Park-Nortown (39)
8644,1295871,43.742343,-79.418292,North York,Bedford Park-Nortown (39)
14589,8000027630,43.735850,-79.420062,North York,Bedford Park-Nortown (39)
14590,8000027630,43.735850,-79.420062,North York,Bedford Park-Nortown (39)
14591,8000027630,43.735850,-79.420062,North York,Bedford Park-Nortown (39)
14762,8001265172,43.739798,-79.421603,North York,Bedford Park-Nortown (39)
14763,8001265172,43.739798,-79.421603,North York,Bedford Park-Nortown (39)
16286,1675722,43.739864,-79.421612,North York,Bedford Park-Nortown (39)


In [34]:
merge_df.sort_values(by='Avg_Severity', ascending=False)

,Combine_Lat_Long,Sum_KSI_Incidents,Sum_HDA_Incidents,Avg_Severity
643,"43.76, -79.18",2,135,0.543000
564,"43.8, -79.29",11,319,0.501333
500,"43.81, -79.21",3,10,0.304000
626,"43.72, -79.29",1,37,0.270000
613,"43.61, -79.5",3,81,0.206667
...,...,...,...,...
250,"43.76, -79.21",9,0,0.000000
571,"43.63, -79.58",3,0,0.000000
244,"43.81, -79.16",7,0,0.000000
243,"43.75, -79.18",1,0,0.000000


In [37]:
ksi[['ACCNUM', 'LATITUDE', 'LONGITUDE', 'DISTRICT', 'STREET1', 'STREET2', 'ACCLASS', 'INJURY']].loc[ksi['Combine_Lat_Long'].isin(['43.76, -79.18'])]

,ACCNUM,LATITUDE,LONGITUDE,DISTRICT,STREET1,STREET2,ACCLASS,INJURY
10771,4001931404,43.760081,-79.182796,Scarborough,MORNINGSIDE AVE,GARDENTREE ST,Non-Fatal Injury,None
10772,4001931404,43.760081,-79.182796,Scarborough,MORNINGSIDE AVE,GARDENTREE ST,Non-Fatal Injury,Major
16537,105298,43.764796,-79.184848,Scarborough,DANZIG ST,MORNINGSIDE AVE,Non-Fatal Injury,None
16538,105298,43.764796,-79.184848,Scarborough,DANZIG ST,MORNINGSIDE AVE,Non-Fatal Injury,Major


In [38]:
ksi[['ACCNUM', 'LATITUDE', 'LONGITUDE', 'DISTRICT', 'STREET1', 'STREET2', 'ACCLASS', 'INJURY']].loc[ksi['Combine_Lat_Long'].isin(['43.8, -79.29'])]

,ACCNUM,LATITUDE,LONGITUDE,DISTRICT,STREET1,STREET2,ACCLASS,INJURY
5921,1148238,43.803745,-79.286290,Scarborough,FINCH Aven E,MIDLAND Aven,Non-Fatal Injury,<Null>
5922,1148238,43.803745,-79.286290,Scarborough,FINCH Aven E,MIDLAND Aven,Non-Fatal Injury,None
5923,1148238,43.803745,-79.286290,Scarborough,FINCH Aven E,MIDLAND Aven,Non-Fatal Injury,Major
6039,1190214,43.803745,-79.286290,Scarborough,FINCH AVE E,MIDLAND AVE,Non-Fatal Injury,None
6228,1190214,43.803745,-79.286290,Scarborough,FINCH AVE E,MIDLAND AVE,Non-Fatal Injury,Major
6229,1190214,43.803745,-79.286290,Scarborough,FINCH AVE E,MIDLAND AVE,Non-Fatal Injury,<Null>
7201,1263305,43.803245,-79.286090,Scarborough,MIDLAND AVE,FINCH AVE E,Non-Fatal Injury,<Null>
7202,1263305,43.803245,-79.286090,Scarborough,MIDLAND AVE,FINCH AVE E,Non-Fatal Injury,<Null>
7203,1263305,43.803245,-79.286090,Scarborough,MIDLAND AVE,FINCH AVE E,Non-Fatal Injury,Major
8893,1321561,43.801845,-79.294290,Scarborough,FINCH AVE E,MILLIKEN BLVD,Non-Fatal Injury,None


In [39]:
ksi[['ACCNUM', 'LATITUDE', 'LONGITUDE', 'DISTRICT', 'STREET1', 'STREET2', 'ACCLASS', 'INJURY']].loc[ksi['Combine_Lat_Long'].isin(['43.81, -79.21'])]

,ACCNUM,LATITUDE,LONGITUDE,DISTRICT,STREET1,STREET2,ACCLASS,INJURY
4002,1085413,43.808345,-79.206490,Scarborough,MCLEVIN Aven,FAWCETT Trai,Non-Fatal Injury,Major
4110,1085413,43.808345,-79.206490,Scarborough,MCLEVIN Aven,FAWCETT Trai,Non-Fatal Injury,None
11376,5001530826,43.814582,-79.213184,Scarborough,SEWELLS RD,HUPFIELD TRL,Non-Fatal Injury,None
11377,5001530826,43.814582,-79.213184,Scarborough,SEWELLS RD,HUPFIELD TRL,Non-Fatal Injury,Minor
11378,5001530826,43.814582,-79.213184,Scarborough,SEWELLS RD,HUPFIELD TRL,Non-Fatal Injury,Major
14890,8000716427,43.814105,-79.213535,Scarborough,SEWELLS RD,HUPFIELD TRL,Fatal,Fatal
